In [1]:
import duckdb
import os
con = duckdb.connect()
con.execute("INSTALL SPATIAL; LOAD SPATIAL")

In [2]:
con.execute("""
CREATE OR REPLACE TABLE aoi AS
SELECT geometry, ST_Transform(geometry, 'EPSG:5070', 'EPSG:4326', always_xy := true) AS t_geom
FROM read_parquet('D:/gis/BaseData/MAV_Boundary.parquet');
""")

In [3]:
import time
t0 = time.perf_counter()
wd = os.getcwd()
print(wd)
# Path to your eBird text file
file_path = os.path.join(wd,'species_csvs','ebd_US_prowar_smp_relSep-2025_sampling.txt')

# Read the file using DuckDB's auto-detection

#con.execute(f"""CREATE TABLE presence AS SELECT * FROM read_csv_auto('{file_path}') WHERE LIMIT 10""")

con.execute(f"""
    CREATE TABLE ebird_points AS
    SELECT 
        'OBSERVATION TYPE', 'OBSERVATION DATE', 'PROTOCOL NAME', ST_Point(LONGITUDE, LATITUDE) AS geometry,
    FROM read_csv_auto('{file_path}');
""")
dt = time.perf_counter() - t0
print(f"Processed files in {dt:.2f}s")

D:\source\statsandmodels\biodiversity\prod


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Processed files in 235.22s


In [7]:
import time
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import os
t0 = time.perf_counter()
wd = os.getcwd()
print(wd)
file_path = os.path.join(wd,'species_csvs','ebd_US_prowar_smp_relSep-2025_sampling.txt')
df = pd.read_csv(file_path, sep='\t') # Adjust separator as needed
geometry = [Point(xy) for xy in zip(df['LONGITUDE'], df['LATITUDE'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326") # Specify appropriate CRS

dt = time.perf_counter() - t0
print(f"Processed files in {dt:.2f}s")

D:\source\statsandmodels\biodiversity\prod


C:\Users\mmitchell\AppData\Local\Temp\ipykernel_48916\823176902.py:10: DtypeWarning: Columns (10,19,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t') # Adjust separator as needed


MemoryError: Unable to allocate 6.16 GiB for an array with shape (9, 91926773) and data type float64

In [101]:
con.execute(f"""describe select * from read_csv_auto('{file_path}')""").df()

,column_name,column_type,null,key,default,extra
0,LAST EDITED DATE,TIMESTAMP,YES,None,None,None
1,COUNTRY,VARCHAR,YES,None,None,None
2,COUNTRY CODE,VARCHAR,YES,None,None,None
3,STATE,VARCHAR,YES,None,None,None
4,STATE CODE,VARCHAR,YES,None,None,None
5,COUNTY,VARCHAR,YES,None,None,None
6,COUNTY CODE,VARCHAR,YES,None,None,None
7,IBA CODE,VARCHAR,YES,None,None,None
8,BCR CODE,BIGINT,YES,None,None,None
9,USFWS CODE,VARCHAR,YES,None,None,None


In [58]:
con.sql("SELECT * from ebird_points limit 10")

┌────────────────────┬────────────────────┬────────────────────────────────┐
│ 'OBSERVATION TYPE' │ 'OBSERVATION DATE' │            geometry            │
│      varchar       │      varchar       │            geometry            │
├────────────────────┼────────────────────┼────────────────────────────────┤
│ OBSERVATION TYPE   │ OBSERVATION DATE   │ POINT (-97.145025 30.008627)   │
│ OBSERVATION TYPE   │ OBSERVATION DATE   │ POINT (-97.145025 30.008627)   │
│ OBSERVATION TYPE   │ OBSERVATION DATE   │ POINT (-97.145025 30.008627)   │
│ OBSERVATION TYPE   │ OBSERVATION DATE   │ POINT (-97.145025 30.008627)   │
│ OBSERVATION TYPE   │ OBSERVATION DATE   │ POINT (-97.145025 30.008627)   │
│ OBSERVATION TYPE   │ OBSERVATION DATE   │ POINT (-89.596664 39.697205)   │
│ OBSERVATION TYPE   │ OBSERVATION DATE   │ POINT (-95.747853 32.218315)   │
│ OBSERVATION TYPE   │ OBSERVATION DATE   │ POINT (-93.7826486 45.2505433) │
│ OBSERVATION TYPE   │ OBSERVATION DATE   │ POINT (-75.7931456 38.9887064) │

In [84]:
aoi = con.execute('select st_astext(t_geom) as geometry from aoi').df()
pts = con.execute('select st_astext(geometry) as geometry from ebird_points limit 10').df()

In [85]:
display(aoi)
display(pts)

,geometry
0,MULTIPOLYGON (((-89.53824232984812 37.26469993...


,geometry
0,POINT (-97.145025 30.008627)
1,POINT (-97.145025 30.008627)
2,POINT (-97.145025 30.008627)
3,POINT (-97.145025 30.008627)
4,POINT (-97.145025 30.008627)
5,POINT (-89.596664 39.697205)
6,POINT (-95.747853 32.218315)
7,POINT (-93.7826486 45.2505433)
8,POINT (-75.7931456 38.9887064)
9,POINT (-75.7931456 38.9887064)


In [87]:
import geopandas as gpd
from shapely import wkt

# Convert WKT strings to shapely geometry objects
aoi['geometry'] = aoi['geometry'].apply(wkt.loads)
# Create a GeoDataFrame with EPSG:4326
aoi_gdf = gpd.GeoDataFrame(aoi, geometry='geometry', crs='EPSG:4326')

pts['geometry'] = pts['geometry'].apply(wkt.loads)
# Create a GeoDataFrame with EPSG:4326
pts_gdf = gpd.GeoDataFrame(pts, geometry='geometry', crs='EPSG:4326')


In [88]:
m = aoi_gdf.explore()
pts_gdf.explore()
m

In [91]:
con.execute("""CREATE OR REPLACE TABLE selected_pts AS
SELECT *
FROM ebird_points
JOIN aoi
ON ST_Intersects(ebird_points.geometry, aoi.t_geom)
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [95]:
display(con.execute('select * from selected_pts limit 10').df())

,'OBSERVATION TYPE','OBSERVATION DATE',geometry,geometry_1,t_geom
0,OBSERVATION TYPE,OBSERVATION DATE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[5, 4, 0, 0, 0, 0, 0, 0, 103, 34, 167, 72, 191...","[5, 4, 0, 0, 0, 0, 0, 0, 87, 159, 184, 194, 14..."
1,OBSERVATION TYPE,OBSERVATION DATE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[5, 4, 0, 0, 0, 0, 0, 0, 103, 34, 167, 72, 191...","[5, 4, 0, 0, 0, 0, 0, 0, 87, 159, 184, 194, 14..."
2,OBSERVATION TYPE,OBSERVATION DATE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[5, 4, 0, 0, 0, 0, 0, 0, 103, 34, 167, 72, 191...","[5, 4, 0, 0, 0, 0, 0, 0, 87, 159, 184, 194, 14..."
3,OBSERVATION TYPE,OBSERVATION DATE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[5, 4, 0, 0, 0, 0, 0, 0, 103, 34, 167, 72, 191...","[5, 4, 0, 0, 0, 0, 0, 0, 87, 159, 184, 194, 14..."
4,OBSERVATION TYPE,OBSERVATION DATE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[5, 4, 0, 0, 0, 0, 0, 0, 103, 34, 167, 72, 191...","[5, 4, 0, 0, 0, 0, 0, 0, 87, 159, 184, 194, 14..."
5,OBSERVATION TYPE,OBSERVATION DATE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[5, 4, 0, 0, 0, 0, 0, 0, 103, 34, 167, 72, 191...","[5, 4, 0, 0, 0, 0, 0, 0, 87, 159, 184, 194, 14..."
6,OBSERVATION TYPE,OBSERVATION DATE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[5, 4, 0, 0, 0, 0, 0, 0, 103, 34, 167, 72, 191...","[5, 4, 0, 0, 0, 0, 0, 0, 87, 159, 184, 194, 14..."
7,OBSERVATION TYPE,OBSERVATION DATE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[5, 4, 0, 0, 0, 0, 0, 0, 103, 34, 167, 72, 191...","[5, 4, 0, 0, 0, 0, 0, 0, 87, 159, 184, 194, 14..."
8,OBSERVATION TYPE,OBSERVATION DATE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[5, 4, 0, 0, 0, 0, 0, 0, 103, 34, 167, 72, 191...","[5, 4, 0, 0, 0, 0, 0, 0, 87, 159, 184, 194, 14..."
9,OBSERVATION TYPE,OBSERVATION DATE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[5, 4, 0, 0, 0, 0, 0, 0, 103, 34, 167, 72, 191...","[5, 4, 0, 0, 0, 0, 0, 0, 87, 159, 184, 194, 14..."


In [104]:
con.execute("""COPY (select * FROM selected_pts limit 50) TO 'testsp.csv' (HEADER, DELIMITER ',');""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))